In [3]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates

import ind.Alpha_base10 as base10


%load_ext autoreload
%autoreload 2
%aimport ind.alpha_base10,tools.Sample_Tools



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# #读取已经缓存的复权数据，并预处理
tail = False
if tail:
    stock_df = load_cache('all_tail_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
else:
    stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)
## 日无风险回报
ret_fs_data = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs_data = (ret_fs_data['high'].astype(np.float32)+ret_fs_data['low'].astype(np.float32))/2 * 0.01
ret_fs_daily = ret_fs_data/252

ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)


#############测试#######################
# ['000001','000008','600600','000729']
# stock_df = stock_df.loc[(slice(None),stock_df.index.get_level_values(1).unique().to_list()[0:11]),:]
# ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)

stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)


def generate_alpha_factors(codes, data_df=None, ret_fs=None, ret=None, ret_excess=None, market_value=None, ret_excess_market=None):
    import ind.alpha_base10 as base10
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd

    df = data_df.loc[(slice(None),codes),:]
    ret_ = ret.loc[(slice(None),codes)]
    ret_excess_ = ret_excess.loc[(slice(None),codes)]
    

    alpha_beta = base10.camp_beta_alpha(ret_excess_,ret_excess_market)
    mom = base10.momentum(ret_,ret_fs)
    size_lg = base10.sizelg(df)
    bp = base10.bp(df)
    
    return pd.concat([alpha_beta, mom, size_lg, bp], axis=1).loc[(slice(None),codes),:]

worker=6
task = task_chunk_split(stock_codes, worker)
results = parallal_task(worker, generate_alpha_factors, task, 
                        data_df=stock_df, ret_fs=ret_fs_daily, ret=ret_t, 
                        ret_excess=ret_t_excess, market_value=market_value_t,
                        ret_excess_market=ret_excess_market_t)
results = pd.concat(results).sort_index()

####使用全局数据的项目，注意不要分步计算
earnings_yield = base10.earnings_yield(ret_t,market_value_t,stock_df['industry']) 
print('earnings_yield')
liquidity = base10.liquidity(stock_df)
print('liquidity')
resvol = base10.resvol(ret_t, ret_fs_daily, ret_t_excess, results['sizelg'], results['beta'], results['residual'])
print('resvol')
sizenl = base10.sizenl(results['sizelg'])

final = pd.concat([results, earnings_yield, liquidity, resvol, sizenl],axis=1).sort_index()
if tail:
    save_cache('base10_tail', final, cache_type=CACHE_TYPE.FACTOR)
else:
    save_cache('base10_train', final, cache_type=CACHE_TYPE.FACTOR)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:15416  pid:3432


  0%|          | 0/6 [00:00<?, ?it/s]

earnings_yield
liquidity
resvol


In [6]:
# load_cache('base10_train',cache_type=CACHE_TYPE.FACTOR).dropna()
# stock_df.loc[(slice(None),['000001','000002']),:][['close']]
stock_df.loc[(slice(None),['000001']),:].shape

(1218, 12)

In [5]:
final.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3022939 entries, (Timestamp('2016-01-04 00:00:00'), '000001') to (Timestamp('2020-12-31 00:00:00'), '605500')
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   beta            float64
 1   alpha           float64
 2   residual        float64
 3   momentum        float64
 4   sizelg          float64
 5   bp              float64
 6   earnings_yield  float64
 7   liquidity       float64
 8   resvol          float64
 9   sizenl          float64
dtypes: float64(10)
memory usage: 242.3+ MB
